In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import os

# ==============================
# 读取数据
# ==============================
print("正在读取数据...")
ratio_df = pd.read_excel("code/FS_RatioStructure.xlsx")
fraud_df = pd.read_excel("data2.xlsx")

print(f"\n比率数据形状: {ratio_df.shape}")
print(f"违规数据形状: {fraud_df.shape}")
print(f"\n比率数据列名（前10列）: {list(ratio_df.columns[:10])}")
print(f"\n违规数据列名: {list(fraud_df.columns)}")

正在读取数据...

比率数据形状: (267302, 361)
违规数据形状: (718, 5)

比率数据列名（前10列）: ['证券代码', '证券简称', '统计截止日期', '流动资产比率', '现金资产比率', '应收类资产比率', '营运资金对流动资产比率', '营运资金比率', '营运资金对净资产比率', '非流动资产比率']

违规数据列名: ['证券代码', '处理文件日期', '证券简称', '违规年度', '上市公司是否违规']


In [2]:
# ==============================
# 构建舞弊标签
# ==============================

print("\n正在构建舞弊集合...")
# 构建舞弊集合
fraud_set = set()
for _, row in fraud_df.iterrows():
    if row["上市公司是否违规"] == "Y":
        symbol = row["证券代码"]
        years_str = str(row["违规年度"])
        # 分割年份，过滤掉非4位数字
        years = [int(y.strip()) for y in years_str.split(";") if y.strip().isdigit() and len(y.strip()) == 4]
        for y in years:
            fraud_set.add((symbol, y))

print(f"✓ 构建了 {len(fraud_set)} 条舞弊记录")

# 提取年份
print("\n正在提取年份...")
ratio_df["统计截止日期"] = pd.to_datetime(ratio_df["统计截止日期"], errors="coerce")
ratio_df["年份"] = ratio_df["统计截止日期"].dt.year
print(f"✓ 年份范围: {ratio_df['年份'].min()} - {ratio_df['年份'].max()}")

# 打标签
print("\n正在打舞弊标签...")
def get_label(row):
    return 1 if (row["证券代码"], row["年份"]) in fraud_set else 0

ratio_df["是否舞弊"] = ratio_df.apply(get_label, axis=1)

print(f"✓ 舞弊样本数: {ratio_df['是否舞弊'].sum()}")
print(f"✓ 非舞弊样本数: {(ratio_df['是否舞弊'] == 0).sum()}")
print(f"✓ 总样本数: {len(ratio_df)}")
print(f"✓ 舞弊比例: {ratio_df['是否舞弊'].mean():.2%}")

# 查看舞弊样本的年份分布
print("\n舞弊样本年份分布:")
if ratio_df["是否舞弊"].sum() > 0:
    print(ratio_df[ratio_df["是否舞弊"] == 1]["年份"].value_counts().sort_index())
else:
    print("⚠️ 警告：未匹配到任何舞弊样本！")
    # 诊断信息
    print("\n🔍 诊断信息：")
    print("比率数据中的证券代码示例（前10个）:", ratio_df["证券代码"].unique()[:10])
    print("违规数据中的证券代码示例（前10个）:", fraud_df["证券代码"].unique()[:10])
    print("舞弊集合示例（前5条）:", list(fraud_set)[:5])


正在构建舞弊集合...
✓ 构建了 1188 条舞弊记录

正在提取年份...
✓ 年份范围: 2015 - 2025

正在打舞弊标签...
✓ 舞弊样本数: 6842
✓ 非舞弊样本数: 260460
✓ 总样本数: 267302
✓ 舞弊比例: 2.56%

舞弊样本年份分布:
年份
2015    398
2016    618
2017    908
2018    858
2019    872
2020    964
2021    916
2022    684
2023    452
2024    163
2025      9
Name: count, dtype: int64


In [3]:
# ==============================
# 特征预处理（缺失值、标准化）
# ==============================

print("\n" + "="*50)
print("开始特征预处理")
print("="*50)

# 定义信息列和特征列
info_cols = ["证券代码", "证券简称", "统计截止日期", "年份", "是否舞弊"]
feature_cols = [col for col in ratio_df.columns if col not in info_cols]

print(f"\n初始特征数量: {len(feature_cols)}")

# 删除缺失值超过25%的列
print("\n正在分析缺失值...")
missing_ratio = ratio_df[feature_cols].isnull().mean()
print(f"✓ 缺失率统计: min={missing_ratio.min():.2%}, max={missing_ratio.max():.2%}, mean={missing_ratio.mean():.2%}")

cols_to_drop = missing_ratio[missing_ratio > 0.25].index.tolist()
cols_to_keep = missing_ratio[missing_ratio <= 0.25].index.tolist()

print(f"✓ 删除高缺失列: {len(cols_to_drop)} 列（缺失率>25%）")
print(f"✓ 保留特征数量: {len(cols_to_keep)} 列")

if len(cols_to_drop) > 0:
    print(f"  被删除的列示例: {cols_to_drop[:5]}...")

# 构建最终数据框
final_df = ratio_df[info_cols + cols_to_keep].copy()

# 中位数填充剩余缺失值
print("\n正在填充缺失值...")
for col in cols_to_keep:
    missing_count = final_df[col].isnull().sum()
    if missing_count > 0:
        final_df[col].fillna(final_df[col].median(), inplace=True)

print(f"✓ 填充完成，剩余缺失值: {final_df[cols_to_keep].isnull().sum().sum()}")

# 标准化特征
print("\n正在标准化特征...")
scaler = StandardScaler()
final_df[cols_to_keep] = scaler.fit_transform(final_df[cols_to_keep])
print(f"✓ 标准化完成（均值≈0，标准差≈1）")

# 保存结果
print("\n正在保存预处理结果...")
output_path = "FS_Preprocessed.xlsx"
final_df.to_excel(output_path, index=False)

print("\n" + "="*50)
print("✅ 预处理完成！")
print("="*50)
print(f"输出文件: {os.path.abspath(output_path)}")
print(f"最终数据形状: {final_df.shape}")
print(f"  - 样本数: {len(final_df)}")
print(f"  - 特征数: {len(cols_to_keep)}")
print(f"  - 舞弊样本: {final_df['是否舞弊'].sum()} ({final_df['是否舞弊'].mean():.2%})")


开始特征预处理

初始特征数量: 358

正在分析缺失值...
✓ 缺失率统计: min=0.00%, max=100.00%, mean=70.51%
✓ 删除高缺失列: 254 列（缺失率>25%）
✓ 保留特征数量: 104 列
  被删除的列示例: ['少数股东权益占比', '主营业务利润占比', '营业外收入占比', '综合税率A', '综合税率B']...

正在填充缺失值...


/var/folders/gt/ffxzg2mx337bvqlsjk85hkqw0000gn/T/ipykernel_53763/2293258611.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df[col].fillna(final_df[col].median(), inplace=True)


✓ 填充完成，剩余缺失值: 0

正在标准化特征...
✓ 标准化完成（均值≈0，标准差≈1）

正在保存预处理结果...

✅ 预处理完成！
输出文件: /Users/cylete/Documents/毕设/辅修毕设/干活/FS_Preprocessed.xlsx
最终数据形状: (267302, 109)
  - 样本数: 267302
  - 特征数: 104
  - 舞弊样本: 6842 (2.56%)
